In [11]:
import csv

def txt_to_csv(input_file, output_file):
    with open(input_file, 'r') as txtfile, open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        for line in txtfile:
            # Dividir la línea utilizando ":::" como separador
            parts = line.strip().split(":::")
            # Escribir las partes en el archivo CSV
            csv_writer.writerow(parts)
   

txt_to_csv('train_data.txt', 'train_data.csv')
txt_to_csv('test_data.txt', 'test_data.csv')
txt_to_csv('test_data_solution.txt', 'test_data_solution.csv')


In [2]:
test_df.head()

,1,2
0,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apar..."
1,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty ch..."
2,Off the Beaten Track (2010),One year in the life of Albin and his family ...
3,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with hi..."
4,Er nu zhai (1955),Before he was known internationally as a mart...


In [3]:
import pandas as pd

train_df = pd.read_csv('train_data.csv', header=None)
test_df = pd.read_csv('test_data.csv', header=None)
# Eliminar primera columna
train_df = train_df.drop(columns=[0])
test_df = test_df.drop(columns=[0])
# Asignar nombres a las columnas: 'title', 'genre', 'plot'
train_df.columns = ['title', 'genre', 'plot']
test_df.columns = ['title', 'plot']
train_df.head()

,title,genre,plot
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [4]:
print("There are {} training samples and {} test samples".format(len(train_df), len(test_df)))

There are 54214 training samples and 54200 test samples


In [5]:
print("There are {} unique genres in the training set".format(train_df['genre'].nunique()))

There are 27 unique genres in the training set


# Text cleaning and processing steps
1. Remove punctuations
2. Convert text to tokens
3. Remove tokens of length less than or equal to 3
4. Remove stopwords using NLTK corpus stopwords list to match
5. Apply stemming
6. Apply lemmatization
7. Convert words to feature vectors

## 1. Remove punctuations

In [17]:
# from "plot" columns remove any punctuation and convert to lowercase
import string as st 
train_df['plot_removed_punct'] = train_df['plot'].str.replace('[{}]'.format(st.punctuation), '').str.lower()
# replace any double-space by single-space
train_df['plot_removed_punct'] = train_df['plot_removed_punct'].str.replace('  ', ' ')
# remove leading and trailing whitespaces
train_df['plot_removed_punct'] = train_df['plot_removed_punct'].str.strip()
train_df.head()

<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
C:\Users\laura\AppData\Local\Temp\ipykernel_6792\2275155347.py:5: SyntaxWarning: invalid escape sequence '\.'
  train_df['plot_removed_punct'] = train_df['plot_removed_punct'].str.replace('\.$', '')


,title,genre,plot,plot_removed_punct,plot_tokenized
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening in to a conversation between his doc...,"[listening, in, to, a, conversation, between, ..."
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...,a brother and sister with a past incestuous re...,"[a, brother, and, sister, with, a, past, inces..."
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,as the bus empties the students for their fiel...,"[as, the, bus, empties, the, students, for, th..."
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,to help their unemployed father make ends meet...,"[to, help, their, unemployed, father, make, en..."
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,the film's title refers not only to the un-rec...,"[the, film's, title, refers, not, only, to, th..."


## 2. Convert text to tokens

In [18]:
# tokenize the plot_removed_punct column using re package
import re
train_df['plot_tokenized'] = train_df['plot_removed_punct'].apply(lambda x: re.split('\s+', x))
train_df.head()

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\laura\AppData\Local\Temp\ipykernel_6792\979581871.py:3: SyntaxWarning: invalid escape sequence '\s'
  train_df['plot_tokenized'] = train_df['plot_removed_punct'].apply(lambda x: re.split('\s+', x))


,title,genre,plot,plot_removed_punct,plot_tokenized
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening in to a conversation between his doc...,"[listening, in, to, a, conversation, between, ..."
1,Cupid (1997),thriller,A brother and sister with a past incestuous r...,a brother and sister with a past incestuous re...,"[a, brother, and, sister, with, a, past, inces..."
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,as the bus empties the students for their fiel...,"[as, the, bus, empties, the, students, for, th..."
3,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,to help their unemployed father make ends meet...,"[to, help, their, unemployed, father, make, en..."
4,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,the film's title refers not only to the un-rec...,"[the, film's, title, refers, not, only, to, th..."


In [20]:
# nltk tokenization
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
train_df['plot_nltk_tokenized'] = train_df['plot_removed_punct'].apply(lambda x: word_tokenize(x))
train_df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laura\AppData\Roaming\nltk_data...


## 3. Remove tokens of length less than or equal to 3

In [ ]:
# Remove tokens of length less than 3

def remove_small_words(text):
    return [x for x in text if len(x) > 3 ]

## 4. Remove stopwords using NLTK corpus stopwords list to match

## 5. Apply stemming

## 6. Apply lemmatization

## 7. Convert words to feature vectors